In [1]:
!nvidia-smi

Fri Apr  5 14:14:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080        On  | 00000000:04:00.0 Off |                  N/A |
|  0%   29C    P8               7W / 180W |    330MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os

# Set the CUDA_VISIBLE_DEVICES environment variable to the index of the GPU you want to use
os.environ["CUDA_VISIBLE_DEVICES"] = "4"  # Set to the index of the GPU you want to use, e.g., "0" for the first GPU

# Now, when you run your code, it will only see the GPU with the specified index


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertModel
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification
import numpy as np
from sklearn.metrics import f1_score
import time
import datetime

/home/hygumm/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-05 14:15:39.308300: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-05 14:15:39.589165: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-05 14:15:40.617848: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared o

In [4]:
import torch
torch.cuda.empty_cache()

In [5]:

# Load the dataset
df1 = pd.read_csv("/home/hygumm/NLP_Project/NTC_SV_train.csv")
df2 = pd.read_csv("/home/hygumm/NLP_Project/NTC_SV_test.csv")
df= pd.concat([df1,df2])

In [6]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [12]:
df.rename(columns = {'review':'text'}, inplace = True) 

In [13]:
df.head()

,text,label
0,đồ_ăn ngon positive hợp_khẩu vị nhiều món nhân...,0
1,chè bơ thơm positive có vị ngậy ngậy nhưng lại...,0
2,chiều hôm nay mới đi ăn về nghe thiên_hạ đồn q...,0
3,mình đặt_hàng qua tin nhắn với cửa_hàng hứa sá...,0
4,ghé mấy lần rồi mà không review đi đâu cũng ch...,1


In [14]:
len(df)

50760

In [15]:
df = df.dropna()

In [16]:
# df = df.sample(n=1000, replace=False)

In [17]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.3)  # 30% for validation

In [18]:

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

In [19]:
df.head()

,text,label
0,đồ_ăn ngon positive hợp_khẩu vị nhiều món nhân...,0
1,chè bơ thơm positive có vị ngậy ngậy nhưng lại...,0
2,chiều hôm nay mới đi ăn về nghe thiên_hạ đồn q...,0
3,mình đặt_hàng qua tin nhắn với cửa_hàng hứa sá...,0
4,ghé mấy lần rồi mà không review đi đâu cũng ch...,1


In [20]:
def preprocess_data(df, tokenizer, max_len=128):
    input_ids = []
    attention_masks = []

    # Drop rows with NaN values
    df = df.dropna()

    for text in df['text']:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(df['label'].values)

    return input_ids, attention_masks, labels

# Preprocess both the training and testing set
train_inputs, train_masks, train_labels = preprocess_data(train_df, tokenizer)
test_inputs, test_masks, test_labels = preprocess_data(val_df, tokenizer)  # Assuming val_df is your test set

In [21]:
batch_size = 16  # Adjust based on your hardware capabilities

# Create the DataLoader for the training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for the test set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [22]:

# Load the BERT model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=2,  # Binary classification
    output_attentions=False,
    output_hidden_states=False,
)

# Specify the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [23]:

optimizer = AdamW(model.parameters(),
                  lr=2e-5,  # Learning rate
                  eps=1e-8)  # Epsilon to prevent division by zero in optimizer

# Number of training epochs
epochs = 4

# Total number of training steps is the number of batches * number of epochs
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,  # Default value
                                            num_training_steps=total_steps)

/home/hygumm/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
# torch.cuda.empty_cache()

In [24]:

# Function to calculate the accuracy of predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Function to format elapsed times as hh:mm:ss
def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Training loop
for epoch_i in range(0, epochs):
    
    # Perform one full pass over the training set.
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes
    t0 = time.time()

    # Reset the total loss for this epoch
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a backward pass
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch)
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        loss = outputs.loss

        # Accumulate the training loss over all of the batches
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients
        loss.backward()

        # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update the learning rate
        scheduler.step()

    # Calculate the average loss over all of the batches
    avg_train_loss = total_train_loss / len(train_dataloader)
    print("\n  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on our validation set

    print("\nRunning Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():        
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)
            
        loss = outputs.loss
        logits = outputs.logits

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(test_dataloader)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("Training complete!")

======== Epoch 1 / 4 ========
Training...
  Batch    40  of  2,221.    Elapsed: 0:00:15.
  Batch    80  of  2,221.    Elapsed: 0:00:27.
  Batch   120  of  2,221.    Elapsed: 0:00:39.
  Batch   160  of  2,221.    Elapsed: 0:00:52.
  Batch   200  of  2,221.    Elapsed: 0:01:04.
  Batch   240  of  2,221.    Elapsed: 0:01:16.
  Batch   280  of  2,221.    Elapsed: 0:01:28.
  Batch   320  of  2,221.    Elapsed: 0:01:41.
  Batch   360  of  2,221.    Elapsed: 0:01:53.
  Batch   400  of  2,221.    Elapsed: 0:02:05.
  Batch   440  of  2,221.    Elapsed: 0:02:18.
  Batch   480  of  2,221.    Elapsed: 0:02:30.
  Batch   520  of  2,221.    Elapsed: 0:02:42.
  Batch   560  of  2,221.    Elapsed: 0:02:55.
  Batch   600  of  2,221.    Elapsed: 0:03:07.
  Batch   640  of  2,221.    Elapsed: 0:03:19.
  Batch   680  of  2,221.    Elapsed: 0:03:32.
  Batch   720  of  2,221.    Elapsed: 0:03:44.
  Batch   760  of  2,221.    Elapsed: 0:03:57.
  Batch   800  of  2,221.    Elapsed: 0:04:09.
  Batch   840  of 

In [27]:
from sklearn.metrics import confusion_matrix

# Initialize lists to store true labels and predicted labels
true_labels = []
pred_labels = []

# Evaluate model on the validation set and collect true and predicted labels
for batch in test_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():        
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)
    
    logits = outputs.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    pred_labels.extend(np.argmax(logits, axis=1).flatten())
    true_labels.extend(label_ids.flatten())

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)

print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[6668  950]
 [ 676 6934]]


In [30]:
from torch.nn.functional import softmax

# Function to predict sentiment label of a sentence
def predict_sentiment(sentence):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    
    # Move inputs to the appropriate device
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # Perform inference (prediction)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Apply softmax to get probabilities
    probabilities = softmax(outputs.logits, dim=1)
    
    # Get the predicted label (0 or 1)
    predicted_label = torch.argmax(probabilities, dim=1).item()
    
    return predicted_label, probabilities

# Example sentence
sentence = "I love this movie!"

# Predict sentiment label
predicted_label, probabilities = predict_sentiment(sentence)
print("Predicted Sentiment Label:", predicted_label)
print("Probabilities:", probabilities)

Predicted Sentiment Label: 1
Probabilities: tensor([[0.0166, 0.9834]], device='cuda:0')


In [31]:
# Example sentence in Vietnamese
example_sentence = "Món ăn ngon và phục vụ tốt!"
#Delicious food and good service!
# Predict sentiment label
predicted_label, probabilities = predict_sentiment(example_sentence)
print("Predicted Sentiment Label:", predicted_label)
print("Probabilities:", probabilities)


Predicted Sentiment Label: 1
Probabilities: tensor([[0.0443, 0.9557]], device='cuda:0')


In [32]:
example_sentence =  "Tôi ghét đồ ăn tôi không thích, tôi sắp nôn mất rồi"
#I hate the food I didnt like it i was about to vomit

In [33]:
# Predict sentiment label
predicted_label, probabilities = predict_sentiment(example_sentence)
print("Predicted Sentiment Label:", predicted_label)
print("Probabilities:", probabilities)

Predicted Sentiment Label: 0
Probabilities: tensor([[0.9985, 0.0015]], device='cuda:0')


In [34]:
# Example sentence
sentence = '''Nothing new....mrunal bad choice....few comedy scenes.....very overacting scenes....ott film not a theatre one .....Vijay overacting spoiled ...
Story is uncooked....funny fights for six feet guy.....
one scene I must tell jagapathi babu first intro ....not even ten seconds 
he give another chance to Vijay for big role with big salary even though 
he rejected his offer very filmy comedy scenes only happens in movie ....very boring second half ......mrunal wake up.....'''

# Predict sentiment label
predicted_label, probabilities = predict_sentiment(sentence)
print("Predicted Sentiment Label:", predicted_label)
print("Probabilities:", probabilities)

Predicted Sentiment Label: 0
Probabilities: tensor([[0.9970, 0.0030]], device='cuda:0')
